# Constraint Programming

**Intuition: Use constraints to reduce the set of values that each variable can take, and Remove values that cannot appear in any solution** Then make a choice when no more deduction can be performend, rinse and repeat.

(compare this to branch and bound, which prunes the search space based on the objective function, not the variables. So this is orthagonal to that approach)

**Modeling Methodology**
- You want to give the solver as much of the problem as possible, the structure of it. Dont go low level.
- Have building blocks to your structure

## "Branch and Prune"
- Branch: decompose problem into subproblems and explore them
- Prune: reduces the search space as much as possible
    - uses constraints to remove from the VARIABLE domains, not from possible objective values

It's a **complete method, not a heuristic**
- given enough time, it will find an optimal solution

The focus is on feasible, how to use variable constraints to determine what is feasible (compare to branch and bound, which is all about relaxing / finding which solutions would be inferior)

## Propagation engine
- Core of any CP solver
- a simple (fixpoint) algorithm
- if infeasible? return failure, else, apply pruning algorithm until no constraint can remove any value, (then you need to test that one)

# Example Problems:

## The 8 Queens Problem:
- Place 8 queens on a chessboard so that none of them attack eachother (same row, column or diagonal)

- You place one. You can then eliminate all of that row/column/diagonal. 
- You have choices for the next one, so repeat
- You can combine with branch and bound, for example if you have a column with only 1 available space left, you KNOW a queen has to go there (b/c 8 columns 8 queens)
- Continue this way until you prove feasible/infeasible, then go back and try different choices

Many possible modelings to this problem, here's one approach:

Associate a decision variable with each column
- The variable denotes the ROW of the queen in that column

Three constraints:
- queens cannot be placed on same row or diagonal
    - So you need to say that row/diag of decision var 1 (col 1) does not equal row/diag of the others


## Coloring a Map

Color a map so that two adjacent territories cannot have the same color

How:
- Choose decision variables
    - The color given to each country
- Express the constraints
    - What's the domain? the set of values a variable can take, the four different colors
    - How do you express? specificy that two adjacent countries cannot have same color (color['G']!=color['B'])

# Illustrating More Complex CP

Important! Every single constraint has its OWN algorithm that will check feasibility and prune
- Feasibility: can a constraint be satifisfied given the values in the domains of the variables
- Pruning: if satisfiable, determine which values in the dommain cannot be a part of any solution

Each algorithm exploits the structure and properties of its constraint

Send More Money example



## Linear Constraints over Integers

Consider the constraint:

a_1*x_1 + ... + a_n*x_n >= b_1*y_1 + ... + b_m*y_m

a_i, b_j >= 0 are constants

x_i, y_j are variables in domain Dx, Dy

First you need to test if it's feasible:

You want to check the left hand side and make it as large as possible and the right side and make it as small as possible. And if those values dont work, then you know it's infeasible.

a_1 * max(D(x_1)) + ... >= b_1 * min(D(y_1)) + ...

let L = max of left side and R = min of right side

Pruning Algorithm:

a_i*x_i >= r - (L - a_i * max(D(x_i)))

therefore: 

x_i >= ceiling(  (r - (L - a_i*max(D(x_i)))) / a_i )

y_i is the same but with l and r switched (and using min), and you use the FLOOR

# Modeling Language of Constraint Programming

## Key Aspect: Ability to state complex, idiosyncratic constraints

### Magic Series

a Series is magic if S_i represents the number of occurences of i in S

S = 2, 1, 2, 0, 0
(two occurences of 0, 1 occurence of 1, 2 occurences of 2, and 0 and 0 of 3 and 4)

Can you find magic series?

```
int n = 5;
range D = 0...n-1
var{int} series[D] in D;
solve {
    forall(k in D)
        series[k] = sum(series[i]=k for (all i in D)
}
```

# Reification: ability to transform a constraint into a 0/1 variables
(1 if constraint is true, 0 otherwise)

System of constraint equations:

series[0] = (series[0]==0)+(series[1]==0) + ...

series[1] = (series[0]==1)+...

So what if series[0]=1?
    You can sub that for your system of equations, reduce, and repeat

# Global Constraints

Capture combinatorial substructures arising in many applications

# Symmetries

Many problems naturally exhibit symmetries. Exploring symmetrical parts are useless.

Many kinds of symmetries (here are 2):
- Variable Symmetries
- Value Symmetries

We'll focus on symmetry-breaking constraints (there are many ways of dealing with constraints)

## Example: Balanced Incomplete Block Designs

Input: (v,b,r,k,l)

Output: a v by b binary matrix with exactly r ones per row, k ones per column, and a scalar product of value l

Plenty of symmetries, if you have a solution, you can swap any two rows or any two columns and still have a solution.

So we dont need to explore all those variations.

How to break variable symmetries:: Impose an ORDERING on the variables

Consider the row symmetries:
- Impose a lexicographical constraint
- In this example, row A is lexo. smaller than B if the most significant value (first ones) are smaller than B

Ex: 0110010 is smaller than 1010100. and 1110010 is bigger than 1010100





# Redundant Constraints

They are semantically redundant (do not exclude any solution)

but they are computationally significant (reduce the search space)

In Constraint programming, you generally want as many constraints as possible

"Can you find a PROPERTY of the solution?"



--- 

Surrogate Constraints

(Make up new constraints that are basically just a combination of existing constraints.)

## Ex. Car sequencing

Manufacturing cars on an assembly line. Cars require specific options (leather seats, moonroof). Capacity constraints on the production units (at most 2 out of 5 cars can require a moonroof)

Goal: sequence all the cars

One trick, work backwards
- If we know that option 1 has the production constraint that only 2/3 cars in a row can have it, and we need 12 cars with it.
- Assume
- That means that in the final 3 slots, we can produce at most 2 cars
- Which means that we need to produce at least 10 cars in the first n-3 slots. and at least 8 cars in the first n-6 slots...

"Look at the right side and see how much you CAN produce, and that tells you how much the left side HAS to produce"

### You can improve your solver by improving communication between your constraints. Global constraints, surrogate constraints, redundant constraints.

You can have TWO models, each with their own decision variable, but you can connect them through constraints.

# How to implement global constraints:
 - napsack and all different

Golden standard for after pruning:
- if value v is in the domain of variable x, then there exists a solution to the constraint with value v assigned to variable x
- Optimal pruning (you cant prune any better wihtout pulling in more information)
- may not be possible in polynomial time

## Binary Knapsack

ex: 10 <= 2x_1 + 3x_2 + 4x_3 + 5x_4 <= 12

Feasibility: can we find a solution to the constraint

Pruning: can we elimnate values from the domains

Feasibility
- Use dynamic programming for Feasibility

Pruning
- Exploit that DP table
- forward phase will keep dependency links
- backward phase, update dependency links to keep only feasible values



## Alldifferent constraint

Feasibility: can we find values in the domains so that each two variables have different values

Pruning: are there values in the domain of a variable that the variable cannot take (i.e. no solution)

Create a bipartite grapth ( a graph with 2 types of verticies, and edges can only exist between different types of vertices)

Here, the vertices are variables/values, and there are edges between them

A matching for a graph is a set of edges such that no two edges share a vertex

a maximum matching is a matching with the largest number of edges

SO: feasibility is equivalent to finding the maximum matching in a bipartite graph

How to find a max matching?
- Start with a matching, then improve it (add edges) if you cant improve, it's the maximum

how to find improvement:
- Find a free vertex
- if there is an edge (x,v) where v is not matched, then insert (x,v) in the matching
- else: take a vertex v matched to y. remove (y,v) and add (x,v). restart at step 2 with Y instead of X

This is an alternating path. an alternating path P for  matching M is a path from vertex X to a vertex v such that the edges in the path are alternatively in E\M and M

Odd number of edges

How to find one: (lecture CP 8: min 22)
- create a directed graph
- given a matching
    - edges in the matchign ar eoriented from bottom to top
    - edges not in the matching are oriented from top to bottom

So an alternating path is starting from a free vertex X, and following the path until you reach a free vertex V

you can do this with depth-first or best-first search, and then do it in time related to number of vertices + edges
